All the sentiment has been stored in IG_posts_with_sentiment.json and the overall score has been made as an equal average of taste, presentation, and creativity scores. Field: "overall_score"

Now, create Gephi bipartite graph (.csv files compatible as nodes and edges in Gephi)

In [ ]:
# All necessary imports

import json
import csv
import pandas as pd

In [3]:
# Load instagram posts

with open('../outputs/IG_posts_with_sentiment.json', 'r', encoding='utf-8') as f:
    posts = json.load(f)

# grab the "Yelp reviewed" restaurants (merged_restaurants_final.csv)

merged_restaurants = pd.read_csv('../outputs/merged_restaurants_final.csv')

# only consider valid_posts -> that is, posts with ratingAvailable = True and restaurant in merged_restaurants
valid_posts = [
    post for post in posts
    if post['ratingAvailable'] == True and post['restaurantName'] in merged_restaurants['name'].values
]

In [5]:
# Track influencers and restaurants

influencer_restaurants = {}
restaurant_influencers = {}

# process all posts

for post in valid_posts:
    influencer = post['influencerUsername']
    restaurant = post['restaurantName']
    
    # track restaurants for each influencer
    if influencer not in influencer_restaurants:
        influencer_restaurants[influencer] = set()
    influencer_restaurants[influencer].add(restaurant)
    
    # track influencers for each restaurant
    if restaurant not in restaurant_influencers:
        restaurant_influencers[restaurant] = set()
    restaurant_influencers[restaurant].add(influencer)


In [ ]:
def write_gephi_files(connections, prefix):
        # Nodes file
        with open(f'{output_dir}/{prefix}_nodes.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Id', 'Label', 'Type'])
            for node in connections.keys():
                writer.writerow([node, node, prefix])
        
        # Edges file
        with open(f'{output_dir}/{prefix}_edges.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Source', 'Target', 'Weight'])
            
            # Create edges between nodes that share connections
            node_list = list(connections.keys())
            for i in range(len(node_list)):
                for j in range(i+1, len(node_list)):
                    # Find common connections
                    common_connections = len(
                        connections[node_list[i]].intersection(connections[node_list[j]])
                    )
                    if common_connections > 0:
                        writer.writerow([node_list[i], node_list[j], common_connections])

